In [1]:
import os
import sys
import json
import datetime
import wandb
import numpy as np
from sklearn.model_selection import train_test_split
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook
from mmengine.visualization import Visualizer, WandbVisBackend
from mmdet.utils import setup_cache_size_limit_of_dynamo
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# mmdetection을 시스템 경로에 추가
sys.path.insert(0, "../mmdetection")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사용자 맞춤형 학습 설정
config_path = './projects/CO-DETR/configs/codino/co_dino_5scale_swin_l_lsj_16xb1_1x_coco.py'  # DINO 설정 파일 경로
work_dir = './work_dirs/co_dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로

In [3]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [4]:
# 학습 및 검증 데이터셋 나누기 (멀티라벨 스트라티파이드 분할)
with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 이미지 ID와 해당 이미지에 포함된 클래스 목록 매핑
image_id_to_classes = {image_id: [] for image_id in image_ids}
for ann in annotations['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    if category_id not in image_id_to_classes[image_id]:
        image_id_to_classes[image_id].append(category_id)

# 멀티라벨 인코딩을 위한 이진 매트릭스 생성
num_images = len(image_ids)
label_matrix = np.zeros((num_images, num_classes))
image_id_to_index = {image_id: idx for idx, image_id in enumerate(image_ids)}
for image_id, class_ids in image_id_to_classes.items():
    idx = image_id_to_index[image_id]
    for class_id in class_ids:
        label_matrix[idx, class_id - 1] = 1  # category_id는 1부터 시작

# 멀티라벨 스트라티파이드 셔플 스플릿 사용
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_indices, val_indices = next(msss.split(image_ids, label_matrix))

# 인덱스를 이미지 ID로 변환
train_ids = [image_ids[idx] for idx in train_indices]
val_ids = [image_ids[idx] for idx in val_indices]

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)

In [5]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)

In [6]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)

# 데이터 전처리기 수정: pad_mask와 pad_seg 비활성화 및 batch_augments 제거
cfg.model.data_preprocessor = dict(
    type='DetDataPreprocessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_size_divisor=32,
    pad_mask=False,       # 마스크 패딩 비활성화
    pad_seg=False,        # 세그멘테이션 패딩 비활성화
    batch_augments=None   # 배치 증강 비활성화
)
    

# 작업 디렉토리 수정
cfg.work_dir = work_dir  # 로그와 모델 저장을 위한 작업 디렉토리 수정

# EarlyStoppingHook 추가
early_stopping_hook = dict(
    type='EarlyStoppingHook',
    monitor='bbox_mAP',
    rule='greater',
    min_delta=0.001,
    patience=5,
    check_finite=True,
    stopping_threshold=None
)

# cfg.default_hooks에 EarlyStoppingHook 추가
cfg.default_hooks.update(
    early_stopping=early_stopping_hook
)

# WandB 초기화 (Runner 전에)
run_name = f'co_dino_5scale_swin_l_lsj_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


WandB Run Initialized: co_dino_5scale_swin_l_lsj_20241024_152536


In [7]:

# 이미지 크기 설정
image_size = (1280, 1280)

cfg.train_dataloader = dict(
    batch_size=1,
    num_workers=2,  # 디버깅을 위해 워커 수를 0으로 설정
    prefetch_factor=2,
    persistent_workers=False,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type='MultiImageMixDataset',  # MultiImageMixDataset 사용
        dataset=dict(
            type='CocoDataset',  # 기본적으로 CocoDataset 사용
            data_root=train_data_root,
            ann_file=train_ann_file,
            data_prefix=dict(img=train_data_root),
            filter_cfg=dict(filter_empty_gt=True, min_size=32),
            pipeline=[
                dict(type='LoadImageFromFile', backend_args=None),
                dict(type='LoadAnnotations', with_bbox=True, with_label=True, with_mask=False),  # with_label=True로 수정
                dict(
                    type='RandomResize',
                    scale=image_size,
                    ratio_range=(0.1, 2.0),  # LSJ를 위한 비율 범위 수정
                    keep_ratio=True
                ),
                dict(
                    type='RandomCrop',
                    crop_type='absolute_range',
                    crop_size=image_size,
                    recompute_bbox=True,
                    allow_negative_crop=True
                ),
                dict(type='FilterAnnotations', min_gt_bbox_wh=(1e-2, 1e-2)),
                dict(type='RandomFlip', prob=0.5),
                dict(type='Pad', size=image_size),
                # dict(type='PackDetInputs')  # 제거
            ],
            metainfo=dict(classes=classes)
        ),
        pipeline=[
            dict(
                type='MixUp',
                img_scale=image_size,        # 이미지 크기 설정
                ratio_range=(0.1, 2.0),      # 비율 범위 수정
                flip_ratio=0.5,
                pad_val=114.0,
                max_iters=15,
                bbox_clip_border=True
            ),
            dict(type='PackDetInputs')  # PackDetInputs를 MixUp 이후에 추가
        ]
    )
)

cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,  # 디버깅을 위해 워커 수를 0으로 설정
    prefetch_factor=2,
    persistent_workers=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',  # 검증 데이터는 MultiImageMixDataset 필요 없음
        data_root=train_data_root,
        ann_file=val_ann_file,
        data_prefix=dict(img=train_data_root),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=[
            dict(type='LoadImageFromFile', backend_args=None),
            dict(type='LoadAnnotations', with_bbox=True, with_label=True, with_mask=False),  # with_label=True로 수정
            dict(
                type='Resize',
                scale=image_size,
                keep_ratio=True
            ),
            dict(type='Pad', size=image_size),
            dict(type='PackDetInputs')  # PackDetInputs는 검증 데이터에서는 필요
        ],
        metainfo=dict(classes=classes)
    )
)

# 데이터 파이프라인 수정: 마스크 관련 부분 제거 및 불필요한 단계 제거
def modify_pipeline(pipeline):
    new_pipeline = []
    for step in pipeline:
        if step['type'] == 'LoadAnnotations':
            step['with_bbox'] = True
            step['with_label'] = True  # with_label=True로 수정
            step['with_mask'] = False  # 마스크 관련 부분 제거
        elif step['type'] == 'FilterAnnotations':
            # FilterAnnotations에서 마스크 관련 설정 제거
            keys_to_remove = ['min_gt_mask_area', 'min_mask_area', 'mask_thr_binary']
            for key in keys_to_remove:
                if key in step:
                    step.pop(key)
        new_pipeline.append(step)
    return new_pipeline

# 파이프라인 수정 적용
cfg.train_dataloader.dataset.dataset.pipeline = modify_pipeline(cfg.train_dataloader.dataset.dataset.pipeline)
# cfg.val_dataloader.dataset.pipeline = modify_pipeline(cfg.val_dataloader.dataset.pipeline)


# 모델 설정에서 mask_head 제거 또는 비활성화
if hasattr(cfg.model, 'mask_head'):
    cfg.model.mask_head = None


# 클래스 수 수정
# bbox_head가 리스트인 경우 모든 헤드에 대해 num_classes 설정
if isinstance(cfg.model.bbox_head, list):
    for head in cfg.model.bbox_head:
        head['num_classes'] = num_classes
else:
    cfg.model.bbox_head['num_classes'] = num_classes

# roi_head 내의 bbox_head도 설정
if isinstance(cfg.model.roi_head, list):
    for roi_head in cfg.model.roi_head:
        if 'bbox_head' in roi_head:
            roi_head['bbox_head']['num_classes'] = num_classes
else:
    if 'bbox_head' in cfg.model.roi_head:
        cfg.model.roi_head['bbox_head']['num_classes'] = num_classes

# query_head도 num_classes 설정
if isinstance(cfg.model.query_head, list):
    for q_head in cfg.model.query_head:
        q_head['num_classes'] = num_classes
else:
    cfg.model.query_head['num_classes'] = num_classes



In [8]:
# 추가: 모델의 모든 관련 헤드에 num_classes가 올바르게 설정되었는지 확인
def verify_num_classes(cfg, num_classes):
    # bbox_head
    if isinstance(cfg.model.bbox_head, list):
        for head in cfg.model.bbox_head:
            assert head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    else:
        assert cfg.model.bbox_head.get('num_classes') == num_classes, "bbox_head num_classes 설정 오류"
    
    # roi_head.bbox_head
    if isinstance(cfg.model.roi_head, list):
        for roi_head in cfg.model.roi_head:
            if 'bbox_head' in roi_head:
                assert roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    else:
        if 'bbox_head' in cfg.model.roi_head:
            assert cfg.model.roi_head['bbox_head'].get('num_classes') == num_classes, "roi_head.bbox_head num_classes 설정 오류"
    
    # query_head
    if isinstance(cfg.model.query_head, list):
        for q_head in cfg.model.query_head:
            assert q_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    else:
        assert cfg.model.query_head.get('num_classes') == num_classes, "query_head num_classes 설정 오류"
    
    print("모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.")

verify_num_classes(cfg, num_classes)

모든 관련 헤드에 대해 num_classes가 올바르게 설정되었습니다.


In [9]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = True  # AMP 활성화
if use_amp:
    cfg.optim_wrapper = dict(
        type='AmpOptimWrapper',          # AMP 옵티마이저 래퍼 사용
        optimizer=cfg.optim_wrapper.optimizer,  # 기존 옵티마이저 설정 유지
        accumulative_counts=1            # 그래디언트 누적 단계
        # scaler 제거: AMP는 내부적으로 자동 처리됩니다.
    )


# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')


In [10]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 1  # 배치 사이즈를 1로 줄임
cfg.val_dataloader.batch_size = 1

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [11]:
# 학습 재개 설정
resume_training = True  # 학습 재개 여부 설정 (True로 설정 시 학습 재개)
resume_checkpoint_path = './work_dirs/co_dino_custom/epoch_9.pth'  # 학습 재개 시 체크포인트 경로 지정

if resume_training:
    if resume_checkpoint_path is None:
        cfg.resume = True
        cfg.load_from = None
    else:
        cfg.resume = False  # 전체 학습 재개를 비활성화 (옵티마이저 상태 로드를 방지)
        cfg.load_from = resume_checkpoint_path  # 모델 가중치만 불러오기
        # 옵티마이저의 학습률을 새로 설정
        cfg.optim_wrapper.optimizer.lr = 1e-5  # 새로운 학습률 설정


In [12]:
# 학습률 설정
cfg.optim_wrapper.optimizer.lr = 1e-5  # 학습률을 1e-5로 설정

In [13]:
# 반복적인 컴파일 횟수를 줄여 학습 속도 향상
setup_cache_size_limit_of_dynamo()

In [14]:
# 평가 설정 추가 및 수정
cfg.evaluation = dict(interval=1, metric='bbox', save_best='auto')

# 평가자(evaluator) 설정 수정
if hasattr(cfg, 'val_evaluator'):
    # 평가자가 딕셔너리 또는 리스트로 정의되어 있을 경우
    if isinstance(cfg.val_evaluator, dict):
        cfg.val_evaluator.ann_file = val_ann_file
    elif isinstance(cfg.val_evaluator, list):
        for evaluator in cfg.val_evaluator:
            evaluator['ann_file'] = val_ann_file
elif 'evaluation' in cfg:
    # 평가 섹션 내에 ann_file을 추가
    cfg.evaluation['ann_file'] = val_ann_file




In [15]:
# # 사전 학습된 모델 경로 설정
# pretrained_model_path = '/data/ephemeral/home/baseline/mmdetection/work_dirs/co_dino_custom/co_dino_5scale_lsj_swin_large_1x_coco-3af73af2.pth'

# # cfg 설정에 사전 학습된 모델 불러오기 추가
# cfg.load_from = pretrained_model_path  # 사전 학습된 모델 경로 지정
# cfg.resume = False  # 학습을 처음부터 시작

# 설정 파일을 통해 러너 생성
runner = Runner.from_cfg(cfg)

# ---------------------- 레이어 동결 및 Unfreeze 설정 시작 ----------------------

# 1. Backbone의 frozen_stages는 이미 설정되어 있으므로 첫 번째 stage는 유지

# 2. 모든 레이어를 동결
for name, param in runner.model.named_parameters():
    param.requires_grad = False

# 3. 필요한 레이어 Unfreeze
# bbox_head Unfreeze
for param in runner.model.bbox_head.parameters():
    param.requires_grad = True

# backbone의 마지막 두 레이어 Unfreeze
for name, param in runner.model.backbone.named_parameters():
    if 'stages.2' in name or 'stages.3' in name:
        param.requires_grad = True

# query_head Unfreeze
for name, param in runner.model.query_head.named_parameters():
    param.requires_grad = True

# roi_head Unfreeze
for name, param in runner.model.roi_head.named_parameters():
    param.requires_grad = True

# neck Unfreeze
for name, param in runner.model.neck.named_parameters():
    param.requires_grad = True

# rpn_head Unfreeze
for name, param in runner.model.rpn_head.named_parameters():
    param.requires_grad = True

# 7. Optimizer 설정 수정
cfg.optim_wrapper.paramwise_cfg = dict(
    custom_keys={
        'backbone.stages.2': dict(lr_mult=0.1),
        'backbone.stages.3': dict(lr_mult=0.1),
        'neck': dict(lr_mult=1.0),
        'query_head': dict(lr_mult=1.0),
        'bbox_head': dict(lr_mult=1.0),
        'roi_head': dict(lr_mult=1.0),
        'rpn_head': dict(lr_mult=1.0)
    }
)

10/24 15:25:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1067258404
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,cod

/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:1325: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  warnings.warn('If you want to reduce GPU memory usage, \
/data/ephemeral/home/baseline/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


10/24 15:26:00 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/24 15:26:00 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

In [16]:
for name, param in runner.model.backbone.named_parameters():
    print(name)


patch_embed.projection.weight
patch_embed.projection.bias
patch_embed.norm.weight
patch_embed.norm.bias
stages.0.blocks.0.norm1.weight
stages.0.blocks.0.norm1.bias
stages.0.blocks.0.attn.w_msa.relative_position_bias_table
stages.0.blocks.0.attn.w_msa.qkv.weight
stages.0.blocks.0.attn.w_msa.qkv.bias
stages.0.blocks.0.attn.w_msa.proj.weight
stages.0.blocks.0.attn.w_msa.proj.bias
stages.0.blocks.0.norm2.weight
stages.0.blocks.0.norm2.bias
stages.0.blocks.0.ffn.layers.0.0.weight
stages.0.blocks.0.ffn.layers.0.0.bias
stages.0.blocks.0.ffn.layers.1.weight
stages.0.blocks.0.ffn.layers.1.bias
stages.0.blocks.1.norm1.weight
stages.0.blocks.1.norm1.bias
stages.0.blocks.1.attn.w_msa.relative_position_bias_table
stages.0.blocks.1.attn.w_msa.qkv.weight
stages.0.blocks.1.attn.w_msa.qkv.bias
stages.0.blocks.1.attn.w_msa.proj.weight
stages.0.blocks.1.attn.w_msa.proj.bias
stages.0.blocks.1.norm2.weight
stages.0.blocks.1.norm2.bias
stages.0.blocks.1.ffn.layers.0.0.weight
stages.0.blocks.1.ffn.layers.0.0

In [17]:

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')

# 학습 시작
runner.train()

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
10/24 15:26:03 - mmengine - INFO - Loads checkpoint by http backend from path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth
Loads checkpoint by local backend from path: ./work_dirs/co_dino_custom/epoch_9.pth
10/24 15:26:06 - mmengine - INFO - Load checkpoint from ./work_dirs/co_dino_custom/epoch_9.pth
10/24 15:26:06 - mmengine - INFO - resumed epoch: 9, iter: 39420
10/24 15:26:06 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
10/24 15:26:06 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
10/24 15:26:06 

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/baseline/mmdetection/../mmdetection/projects/CO-DETR/codetr/transformer.py:991: UserWarning: __floordiv__ is deprecated, and its behavio

10/24 15:27:33 - mmengine - INFO - Epoch(train) [10][  50/4380]  lr: 1.0000e-04  eta: 3 days, 15:07:05  time: 1.7469  data_time: 0.0194  memory: 23790  loss: 40.0350  loss_cls: 0.4855  loss_bbox: 0.1591  loss_iou: 0.2376  d0.loss_cls: 0.6312  d0.loss_bbox: 0.1796  d0.loss_iou: 0.2624  d1.loss_cls: 0.5405  d1.loss_bbox: 0.1594  d1.loss_iou: 0.2403  d2.loss_cls: 0.5128  d2.loss_bbox: 0.1527  d2.loss_iou: 0.2324  d3.loss_cls: 0.4998  d3.loss_bbox: 0.1567  d3.loss_iou: 0.2335  d4.loss_cls: 0.4893  d4.loss_bbox: 0.1568  d4.loss_iou: 0.2348  enc_loss_cls: 0.5809  enc_loss_bbox: 0.2117  enc_loss_iou: 0.2998  dn_loss_cls: 0.0892  dn_loss_bbox: 0.3515  dn_loss_iou: 0.3391  d0.dn_loss_cls: 0.1440  d0.dn_loss_bbox: 0.5166  d0.dn_loss_iou: 0.5008  d1.dn_loss_cls: 0.0972  d1.dn_loss_bbox: 0.3832  d1.dn_loss_iou: 0.3755  d2.dn_loss_cls: 0.0900  d2.dn_loss_bbox: 0.3552  d2.dn_loss_iou: 0.3471  d3.dn_loss_cls: 0.0901  d3.dn_loss_bbox: 0.3513  d3.dn_loss_iou: 0.3388  d4.dn_loss_cls: 0.0891  d4.dn_loss_

KeyboardInterrupt: 

In [35]:
print("Train Pipeline:")
for step in cfg.train_dataloader.dataset.pipeline:
    print(step)

print("Validation Pipeline:")
for step in cfg.val_dataloader.dataset.pipeline:
    print(step)


Train Pipeline:
{'type': 'LoadImageFromFile', 'backend_args': None}
{'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': False}
{'type': 'RandomResize', 'scale': (1024, 1024), 'ratio_range': (0.8, 1.25), 'keep_ratio': True}
{'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (1024, 1024), 'recompute_bbox': True, 'allow_negative_crop': True}
{'type': 'FilterAnnotations', 'min_gt_bbox_wh': (0.01, 0.01)}
{'type': 'RandomFlip', 'prob': 0.5}
{'type': 'Pad', 'size': (1024, 1024)}
{'type': 'PackDetInputs'}
Validation Pipeline:
{'type': 'LoadImageFromFile', 'backend_args': None}
{'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': False}
{'type': 'RandomResize', 'scale': (1024, 1024), 'ratio_range': (0.8, 1.25), 'keep_ratio': True}
{'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (1024, 1024), 'recompute_bbox': True, 'allow_negative_crop': True}
{'type': 'FilterAnnotations', 'min_gt_bbox_wh': (0.01, 0.01)}
{'type': 'RandomFlip', 'prob': 0.5}
{'typ